# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

Jay: OpenAI(): This is likely the standard, synchronous version. When you call a method on an instance of this class, the method runs to completion and returns a result. Your program waits for the method to finish before continuing.

AsyncOpenAI(): This is likely the asynchronous version. When you call a method on an instance of this class, the method starts running but your program doesn't wait for it to finish. Instead, the method returns a "future" object that represents the result of the method. Your program can continue doing other things and check the result of the method later.

In general, asynchronous programming can make your program more efficient by allowing it to do other things while waiting for slow operations (like network requests) to complete. However, it can also make your program more complex because you have to manage the "futures" and handle the possibility that operations might not complete in the order you started them.

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [ ]:
!pip install -qU openai

In [ ]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

OpenAI API Key:··········


## Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [1]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [2]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "Poppy" # @param {type: "string"}
instructions = "You are a helpful FAQ assistant specializing in pediatric dentistry. Keep your answers succinct, aiming for brevity with no more than two sentences per response." # @param {type: "string"}
model = "gpt-4-turbo" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4"]

Jay: You will embody Poppy, an AI chat assistant specializing in pediatric dentistry FAQs. Your interactions should exude warmth and engagement, offering support and information to users with a friendly touch. Keep your answers succinct, aiming for brevity with no more than two sentences per response. Should a question fall outside your expertise, kindly inform the user, 'I’m sorry, but I can only provide information related to pediatric dentistry.' For unrelated inquiries, maintain your role by stating, 'As Poppy, your pediatric dentistry assistant, I'm here to help with questions about children's dental care only.' Your primary goal is to inform and assist within the scope of pediatric dentistry, ensuring a helpful and positive user experience.


### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [3]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [4]:
assistant

Assistant(id='asst_eGgBBayHusd016cplTLvOJlr', created_at=1712893835, description=None, file_ids=[], instructions='You are a helpful FAQ assistant specializing in pediatric dentistry. Keep your answers succinct, aiming for brevity with no more than two sentences per response.', metadata={}, model='gpt-4-turbo', name='Poppy', object='assistant', tools=[])

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [5]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [6]:
thread

Thread(id='thread_w2UnIiuXcXI4IoWCAxQxzeLT', created_at=1712927206, metadata={}, object='thread')

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [8]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"Why should I choose a pediatric dentist for my child over a geenral dentist?"
)

Again, let's examine our `message` object!

In [9]:
message

Message(id='msg_RdsX8Jw76gkYX8Cuq7QwwTDT', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Why should I choose a pediatric dentist for my child over a geenral dentist?'), type='text')], created_at=1712927318, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_w2UnIiuXcXI4IoWCAxQxzeLT')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

Jay: Analyze the question and chunks to determine if clarification is needed:

Check query specificity and chunk scope.
If query is broad or lacks essential details, output '#' and provide a clear explanation of the information needed for a comprehensive answer.
If query matches a specific chunk that provides a complete answer, output '~'.
Output: [Decision: '~' or '#'. If '#', provide a detailed clarification request specifying the information needed to answer the question effectively. Tailor the clarification to the specific query and context.]

Example:
Question: "Should my child use an electric toothbrush?"
Output: [Decision: '#', Please provide the age of your child and any specific dental concerns or needs they may have. This information will help determine if an electric toothbrush is suitable for them.]

Note: For age-specific or child's dental situation questions, always ask for clarification if essential details are not provided.

In [10]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = "Your audience is the parents of children. Think step by step and sound confident in your replies." # @param {type: "string"}

Let's run our Thread!

In [11]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=instructions + " " + additional_instructions
)

In [12]:
#Jay:
print(instructions + " " + additional_instructions)

You are a helpful FAQ assistant specializing in pediatric dentistry. Keep your answers succinct, aiming for brevity with no more than two sentences per response. Your audience is the parents of children. Think step by step and sound confident in your replies.


Now that we've run our thread, let's look at the object!

In [13]:
run

Run(id='run_Nq5fLRcpo49bkXzEgR7m6a9M', assistant_id='asst_eGgBBayHusd016cplTLvOJlr', cancelled_at=None, completed_at=None, created_at=1712928534, expires_at=1712929134, failed_at=None, file_ids=[], instructions='You are a helpful FAQ assistant specializing in pediatric dentistry. Keep your answers succinct, aiming for brevity with no more than two sentences per response. Your audience is the parents of children. Think step by step and sound confident in your replies.', last_error=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_w2UnIiuXcXI4IoWCAxQxzeLT', tools=[], usage=None, temperature=1.0)

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [14]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [15]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [16]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [17]:
messages.data[0]

Message(id='msg_1MW5RPPaUer8JzmZyMSGq4od', assistant_id='asst_eGgBBayHusd016cplTLvOJlr', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Pediatric dentists specialize in the dental development and specific needs of children, receiving two to three years of additional training focused on treating young patients. They are also equipped to provide a comfortable, child-friendly environment that eases the anxiety associated with dental visits.'), type='text')], created_at=1712928537, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Nq5fLRcpo49bkXzEgR7m6a9M', status=None, thread_id='thread_w2UnIiuXcXI4IoWCAxQxzeLT')

## Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Creating an Assistant with the Retriever Tool

The first thing we'll want to do is create an assistant with the Retriever tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data

First, we need some data. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [ ]:
!wget https://www.gutenberg.org/files/84/84-h/84-h.htm -o frankenstein.html

## Find and download my corpus from local
Jay: I manually added my corpus. My content is stored in Webflow CMS - a web design app. Ideally I would make a call to the CMS to download my content but I did not not want to work on that piece to stay in scope for this exercise. faq.txt.

In [20]:
import os

def find_file(filename, search_path):
    for root, dir, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)

# Replace 'myfile.txt' with your file name and '/' with your search path
file_path = find_file('faqv3.txt', '/')
print(file_path)

/System/Volumes/Data/Users/acrobat/Desktop/Voiceflow/knowledge base/V3/faqv3.txt


In [22]:
!cp "/System/Volumes/Data/Users/acrobat/Desktop/Voiceflow/knowledge base/V3/faqv3.txt" faqv3.txt

Now we can upload our file!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [51]:
file_reference = client.files.create(
  file=open("faqv3.txt", "rb"),
  purpose='assistants'
)

Let's look at what our `file_reference` contains!

In [24]:
file_reference

FileObject(id='file-OeOr9biNmtGwwkjbkcp8xlq1', bytes=11043, created_at=1712932251, filename='faqv3.txt', object='file', purpose='assistants', status='processed', status_details=None)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Please pay attention to [pricing](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) and don't forget to delete your files when you're done!

In [ ]:
Here we are also updating the name appending + Retrieval to the name of the assistant. mine is Poppy + Retrieval. Here also assigning the rerieval tool using tools...

In [25]:
assistant = client.beta.assistants.create(
  name=name + "+ Retrieval",
  instructions=instructions,
  model=model,
  tools=[{"type": "retrieval"}],
  file_ids=[file_reference.id]
)

Let's try submitting a message to our Assistant and seeing what kind of answer we get!

We'll outline the steps needed to do this in full:

1. Create an Assistant
2. Create a Thread
3. Add Messages to that Thread
4. Create a Run on that Thread
5. Wait for Run to Complete
6. Collect Messages from the Thread

Let's do that below!

In [26]:
# Create a Thread
thread = client.beta.threads.create()

# Add Messages to that Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What to do if my child's tooth is wiggly?"
)

# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress


Let's look at the final result!

In [27]:
messages

SyncCursorPage[Message](data=[Message(id='msg_LXerNK4rSONzpOr19rLaAEEz', assistant_id='asst_sK66WSETvfQiRP7RlXzYMCWB', completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=312, file_citation=FileCitation(file_id='file-OeOr9biNmtGwwkjbkcp8xlq1', quote=''), start_index=302, text='【5†source】', type='file_citation')], value="If your child has a wiggly tooth, encourage them to gently wiggle it but avoid forcing it out before it's ready, as this can cause pain and increase the risk of infection. If the loose tooth is causing discomfort or there are concerns about its progression, consider consulting with a pediatric dentist【5†source】."), type='text')], created_at=1712943837, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_KWmqy4QJbXAeWL6mD2uuDPRL', status=None, thread_id='thread_XD2WT6Gpkha7tXKr30CW61I3'), Message(id='msg_26ppltFlv94xJHYfSGP6AKJm', assistant_id=None, 

Let's do some clean up to make sure we're not being charged anything extra by deleting our resources.

In [43]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

### Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [52]:
assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
  #file_ids=[file_reference.id]
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [53]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "file_ids": [file_reference.id]
    }
  ]
)

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [54]:
# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress


We can check the specific steps that the Code Interpreter ran to figure out what steps the Assistant took!

In [55]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [56]:
for step in run_steps.data:
  print(step.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_wq3MyjFuTsqqHqeJsyxxxOZu'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_B9Vecod4LcRnbJ0eKlxznDdO', code_interpreter=CodeInterpreter(input="import mimetypes\r\n\r\n# Identify the MIME type of the uploaded file\r\nfile_path = '/mnt/data/file-dTXnmJy2IJCxjBiQHNAblQuE'\r\nmime_type, _ = mimetypes.guess_type(file_path)\r\nmime_type", outputs=[CodeInterpreterOutputLogs(logs='', type='logs')]), type='code_interpreter')], type='tool_calls')


In [57]:
messages

SyncCursorPage[Message](data=[Message(id='msg_wq3MyjFuTsqqHqeJsyxxxOZu', assistant_id='asst_fGDdS7H1qAMpgz2RpYOjVTde', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The MIME type of the uploaded file could not be determined. It might be necessary to inspect the content or extension of the file for additional clues.'), type='text')], created_at=1712948774, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_qnJHDTgiXW6xi0uRjKa1D6g6', status=None, thread_id='thread_cLpgd9xGAF2xVLmYRfWtzwb8'), Message(id='msg_XgL79pmNQLVspyJX5Pzspy3B', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What kind of file is this?'), type='text')], created_at=1712948762, file_ids=['file-dTXnmJy2IJCxjBiQHNAblQuE'], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_cLpgd9

In [58]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [59]:
!pip install -qU duckduckgo_search


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [60]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [63]:
duckduckgo_search("How many 5 star Google reviews does a pediatric dentistry need to place at top of Google search results?")

'A study by Harvard Business School found that a 1-star increase in Yelp ratings leads to a 5-9% increase in revenue. And while a Google review may not have as much of an impact, they can still lead to more sales and revenue for your business. So, if you want to improve your local SEO, build trust and credibility, and generate more sales, ensure ...\nHow to use the calculator correctly. The 5-star rating calculator is a handy little tool that can help you figure out how many more reviews you need to get your overall or average rating to 5 stars. Here are a few examples of how to use it correctly: 1. Enter the current number of reviews you have. 2. Enter the current average rating you have. 3.\nSimilar to other types of reviews, recipe cards in search results show the average review rating and the total number of reviews. Screenshot from search for [best vegan winter recipes], Google ...\nA 5-star rating can be what you need to stand out in search results. Trust and SEO: Positive Google

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [65]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'What is the importance of Google reviews in local SEO?'"
            }
        },
        "required" : ["query"]
    }
}

####❓ Question

Why does the description key-value pair matter?

#### ANSWER:
It helps clarify the purpose and usage of the function. 

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

Jay: Can I assign multiple functions simultanously?

In [66]:
assistant = client.beta.assistants.create(
    name=name + " + Function Calling API",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

We need to make a few modifications to our Assistant to include the ability to make calls to our local function and pass the results back to our Assistant for further generation.

In [68]:
import json

def wait_for_run_completion(thread_id, run_id):
    """
    Waits for the completion of a run identified by the given thread ID and run ID.

    Args:
        thread_id (str): The ID of the thread containing the run.
        run_id (str): The ID of the run to wait for.

    Returns:
        dict: The details of the completed run.

    Raises:
        None

    """
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

def submit_tool_outputs(thread_id, run_id, tools_to_call):
    """
    Submits the tool outputs to the specified thread and run.

    Args:
        thread_id (str): The ID of the thread to submit the tool outputs to.
        run_id (str): The ID of the run to submit the tool outputs to.
        tools_to_call (list): A list of tools to call and retrieve outputs from.

    Returns:
        dict: A dictionary containing the submitted tool outputs.

    """
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "duckduckgo_search":
            print("Consulting Duck Duck Go...")
            output = duckduckgo_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    print(tool_output_array)

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

def print_messages_from_thread(thread_id):
    """
    Prints the messages from a given thread.

    Args:
        thread_id (str): The ID of the thread.

    Returns:
        None
    """
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

def use_assistant(query, assistant_id, thread_id=None):
    """
    Uses the OpenAI Assistant to interact with the AI model.

    Args:
        query (str): The user's query or message.
        assistant_id (str): The ID of the OpenAI Assistant.
        thread_id (str, optional): The ID of the thread to use. If not provided, a new thread will be created.

    Returns:
        str: The ID of the thread used for the conversation.
    """
    thread = client.beta.threads.create()

    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=query,
    )

    print("Creating Assistant ")

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

    print("Querying OpenAI Assistant Thread.")

    run = wait_for_run_completion(thread.id, run.id)

    if run.status == 'requires_action':
        run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
        run = wait_for_run_completion(thread.id, run.id)

    print_messages_from_thread(thread.id)

    return thread.id


####❓ Question

Outline, in simple terms, what the `use_assistant` helper function is doing.

#### ANSWER
The use_assistant function is the main function that uses the assistant. It creates a new thread, creates a new message in the thread with the user's query, and creates a new run with the assistant. If the run requires action, it submits the tool outputs and waits for the run to complete. Finally, it prints all the messages from the thread and returns the thread id.

In [69]:
use_assistant("Do I need Goggle reviews for local SEO?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_nDMnWA1z7x8MZvoihT5zGnVs', 'output': "1) The Owner Sets The Review Stage. 65% of review writers have left negative reviews after experiencing bad or rude customer service, and an additional 28% say their negative reviews result from businesses failing to resolve their complaints at the time of service. One of the best paths to building a sterling local business reputation is to run ...\nPlus, skillful review management can also impact a business's success. Hence, you may have already got a gist of why are Google Reviews important. Now, let's dive into the 7 staggering benefits of Google Reviews in detail! 1. Increases Trust and Credibility. Google dominates the search engine market with a 92% share.\nHow Reviews & Ratings Impact Local SEO for Google Maps. For listings on Google Maps, our local rank trackers found th

'thread_esMzUcQOK8dh25E6HvpTdxzV'

## Wrapping it All Together (Super good)

Now we can create an Assistant with all of the available tools and see how it responds to various queries!

In [70]:
assistant = client.beta.assistants.create(
    name=name + " + All Tools",
    instructions=instructions,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function" : ddg_function}
    ],
    model=model,
    file_ids=[file_reference.id],
)

In [71]:
use_assistant("Why should I choose a pediatric dentist over a general dentist?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: Choosing a pediatric dentist over a general dentist is recommended because pediatric dentists receive specialized training in caring for children’s teeth, gums, and mouth through all stages of childhood. They are also trained to handle the unique behavioral needs of children, making dental visits more positive and effective for young patients.
user: Why should I choose a pediatric dentist over a general dentist?


'thread_eEcsOtjN1MJb6xMlj07cpvLe'

In [72]:
use_assistant("How many questions are there in my document?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The document contains a total of 11 questions.
user: How many questions are there in my document?


'thread_W1oO6LbADHBSQ02WNMF1j8fI'

In [73]:
use_assistant("Extract all questions in my document as  bullet points. I only want the questions.", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: - When is a good time for a child's first dental visit and what should parents expect?
- What age should children start getting dental x-rays and why are they important?
- What's the best age to introduce an electric toothbrush to children and which type is recommended?
- Why do children's teeth sometimes appear yellow and how can it be addressed?
- How can parents determine the right time for their child to visit an orthodontist?
- What age do children typically lose their first tooth and what should parents do when their child's tooth is wiggly?
- Until what age does your pediatric dental practice provide ca

'thread_OrnTmB0SnJ042iKZPisRsWaH'

####❓ Question

Notice that our response can go through multiple paths, given that:

What is "deciding" to use the tool?

#### ANSWER:
I think the answer is both sequential and contextual logic. This makes the most sense to me. I think this is the reason we add answer non technical question. The model checks the knowledge base first (retrival). If it does not find the answer there then moves on to the web search. This is controlled by the "non technical" question part. if there is any math / coding involved it defaults to the code retrieval tool.
One interesting thing would be to try to add the coding questions answer to the knowledge base. In my example I have 13 questions with answers in my document. What if one of my question asked: How many questions are there in my document? And the answer was 20 instead of the correct 13. Would it return 20 since it finds the answer in my knowledge base?

### Adding JSON Mode for More Agentic Behaviour

Finally, we have the ability to select tools - all we need to do now is set up a process to allow us to create some kind of loop and make decisions about whether or not the response is complete or not.

We'll leverage the OpenAI completions end-endpoint with JSON mode to let us understand when we've adequately answered our user's question!

In [74]:
completed_template = \
"""
Does this response adequately answer the user's query?

Please return your response in JSON format - with key: "completed" and either True (if completed) or False (if not completed)

User Query:
{query}

Assistant Response:
{response}
"""

def is_complete(query, response):
  completed_response = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": completed_template.format(query=query, response=response),
          }
      ],
      model=model,
      response_format={"type" : "json_object"}
  )

  return completed_response

In [88]:
query = "How many bytes is the provided file?"

thread_id_for_response = use_assistant(query, assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The provided file is 11,043 bytes in size.
user: How many bytes is the provided file?


Now we can observe JSON mode in action!

In [76]:
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
response = messages.data[0].content[0].text.value
completed_flag = json.loads(is_complete(query, response).choices[0].message.content)

In [84]:
messages

SyncCursorPage[Message](data=[Message(id='msg_t1QJ4ozNngEAkA7YxWcFdddj', assistant_id='asst_dYx26lafARyFiozBz5MfFNZg', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The provided file is 11,043 bytes in size.'), type='text')], created_at=1712951728, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_O60vqxyOrMBAkfUWrR0SPbjD', status=None, thread_id='thread_uDRImtznWXjo2SjeKSuwSCuP'), Message(id='msg_YgKDf549KUJb9AjbDGLSfL9X', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How many bytes is the provided file?'), type='text')], created_at=1712951723, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_uDRImtznWXjo2SjeKSuwSCuP')], object='list', first_id='msg_t1QJ4ozNngEAkA7YxWcFdddj', last_id='msg_YgKDf549KUJb9AjbDGLSfL9X', has_more=Fa

In [85]:
response

'The provided file is 11,043 bytes in size.'

In [78]:
completed_flag

{'completed': True}


## 🚧 BONUS CHALLENGE 🚧:

Use the components we've constructed so far to build a loop that lets us continue to query the Assistant if the response is not completed!

In [91]:
### YOUR CODE HERE: not sure how to test this other then this. Sometimes it runs for 5 cycles, sometimes just twice. 


query = "What is in my pocket right now?"
completed_flag = False

while completed_flag == False:
    thread_id_for_response = use_assistant(query, assistant.id)
    messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
    response = messages.data[0].content[0].text.value
    completed_flag = json.loads(is_complete(query, response).choices[0].message.content)['completed']


Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: I don't have access to information about what's currently in your pocket. If you have a different question, particularly about pediatric dentistry or something related to a document you've uploaded, I'd be happy to help with that!
user: What is in my pocket right now?
Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: I don't have the ability to know or detect what is physically in your pocket. If this is related to the content of a document or file you've uploaded, please specify or ask a related question to that content.
user: What is in my pocket right now?


In [92]:
completed_flag

True

# Make Sure You Delete Resources

Make sure you delete all the resources you created!

This function will help you do so!

In [ ]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)